# EDA

## Data reading and input

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import pickle
from sklearn import preprocessing
from scipy.stats import chi2_contingency
from scipy.stats.contingency import association
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, KFold, GridSearchCV
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
import time
from tqdm.notebook import tqdm
!pip install imbalanced-learn
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Machine Learning Datasets/UTS/reguler_fkrtl.csv')
df.head()

/tmp/ipython-input-3872584673.py:1: DtypeWarning: Columns (43,46,49,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Machine Learning Datasets/UTS/reguler_fkrtl.csv')


,Unnamed: 0,PSTV01,PSTV02,PSTV15,FKP02,FKL02,FKL03,FKL04,FKL05,FKL06,...,FKL39,FKL40,FKL41,FKL42,FKL43,FKL44,FKL45,FKL46,FKL47,FKL48
0,0,455297864,354398080,10.273760,NaN,134540221V000105,02/26/2021,02/28/2021,DAERAH ISTIMEWA YOGYAKARTA,KOTA YOGYAKARTA,...,NaN,0,NaN,NaN,0,NaN,NaN,0,3426200,3426200
1,1,428331611,355534480,248.677670,NaN,459250521V000580,05/31/2021,06/02/2021,JAWA TIMUR,SIDOARJO,...,NaN,0,NaN,NaN,0,NaN,NaN,0,2155300,2155300
2,2,159537057,159537057,36.873344,NaN,242680121V000408,01/04/2021,01/07/2021,ACEH,KOTA LHOKSEUMAWE,...,NaN,0,NaN,NaN,0,NaN,NaN,0,2541200,2541200
3,3,87132631,98569517,92.235886,NaN,112271221V002764,12/20/2021,12/25/2021,JAWA TENGAH,GROBOGAN,...,NaN,0,NaN,NaN,0,NaN,NaN,0,2586400,2586400
4,4,25031617,14211084,78.579094,NaN,321561121V004514,11/21/2021,11/25/2021,JAWA BARAT,BANDUNG,...,NaN,0,NaN,NaN,0,NaN,NaN,0,2586400,2586400


In [ ]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 872201 entries, 0 to 872200
Data columns (total 56 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  872201 non-null  int64  
 1   PSTV01      872201 non-null  int64  
 2   PSTV02      872201 non-null  int64  
 3   PSTV15      872201 non-null  float64
 4   FKP02       519692 non-null  object 
 5   FKL02       872201 non-null  object 
 6   FKL03       872201 non-null  object 
 7   FKL04       872201 non-null  object 
 8   FKL05       872201 non-null  object 
 9   FKL06       872201 non-null  object 
 10  FKL07       872201 non-null  object 
 11  FKL08       872201 non-null  object 
 12  FKL09       872201 non-null  object 
 13  FKL10       872201 non-null  object 
 14  FKL11       872201 non-null  object 
 15  FKL12       872201 non-null  object 
 16  FKL13       872201 non-null  object 
 17  FKL14       872201 non-null  object 
 18  FKL15       872201 non-null  object 
 19  FK

,Unnamed: 0,PSTV01,PSTV02,PSTV15,FKL22,FKL32,FKL34,FKL37,FKL40,FKL43,FKL46,FKL47,FKL48
count,872201.000000,8.722010e+05,8.722010e+05,872201.000000,872201.000000,8.722010e+05,8.722010e+05,8.722010e+05,8.722010e+05,8.722010e+05,8.722010e+05,8.722010e+05,8.722010e+05
mean,436100.000000,1.407332e+08,1.404167e+08,79.515331,30.807534,8.372282e+05,3.620109e+02,2.066690e+04,2.269116e+03,1.603546e+03,9.526411e+03,8.717836e+05,8.706864e+05
std,251782.885405,1.331351e+08,1.290404e+08,170.802420,14.135539,2.614295e+06,6.692782e+04,3.791299e+05,2.198844e+05,5.480110e+04,2.552267e+05,2.773236e+06,2.766543e+06
min,0.000000,1.144000e+03,2.147000e+03,0.504971,1.000000,6.820000e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.820000e+04,0.000000e+00
25%,218050.000000,4.129074e+07,4.347448e+07,10.400174,16.000000,1.849000e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.849000e+05,1.849000e+05
50%,436100.000000,8.357546e+07,8.744769e+07,33.826828,42.000000,1.922000e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.922000e+05,1.922000e+05
75%,654150.000000,2.236668e+08,2.159169e+08,90.344950,44.000000,3.959000e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.959000e+05,3.959000e+05
max,872200.000000,4.605538e+08,4.605247e+08,18235.559000,84.000000,3.125353e+08,1.843370e+07,4.974100e+07,1.007643e+08,3.332900e+06,1.586110e+07,3.125353e+08,3.125353e+08


In [ ]:
# Memeriksa inkonsistensi format
object_cols = df.select_dtypes(include='object').columns

for col in object_cols:
    print(f"Unique values in '{col}':")
    print(df[col].unique())
    print("-" * 30)

Unique values in 'FKP02':
[nan '72051121P000720' '52480621P000004' ... '438001021P000102'
 '254451021Y000063' '48131121P000076']
------------------------------
Unique values in 'FKL02':
['134540221V000105' '459250521V000580' '242680121V000408' ...
 '181620321V007152' '75310321V000082' '329770221V002128']
------------------------------
Unique values in 'FKL03':
['02/26/2021' '05/31/2021' '01/04/2021' '12/20/2021' '11/21/2021'
 '08/25/2021' '08/13/2021' '03/17/2021' '11/15/2021' '01/11/2021'
 '09/26/2021' '05/22/2021' '02/22/2021' '11/11/2021' '06/14/2021'
 '03/11/2021' '09/23/2021' '01/13/2021' '12/29/2020' '06/30/2021'
 '10/06/2021' '03/30/2021' '12/24/2021' '01/03/2021' '05/17/2021'
 '05/28/2021' '08/20/2021' '06/24/2021' '07/27/2021' '09/19/2021'
 '01/01/2021' '11/16/2021' '03/05/2021' '04/08/2021' '03/29/2021'
 '01/30/2021' '03/21/2021' '06/02/2021' '12/16/2021' '11/10/2021'
 '01/07/2021' '05/14/2021' '12/04/2021' '04/03/2021' '12/14/2021'
 '06/28/2021' '07/20/2021' '09/14/2021' '12

In [ ]:
df.isnull().sum()

,0
Unnamed: 0,0
PSTV01,0
PSTV02,0
PSTV15,0
FKP02,352509
FKL02,0
FKL03,0
FKL04,0
FKL05,0
FKL06,0


## Data Cleaning

In [ ]:
df.drop(['FKP02', 'FKL02','FKL03','FKL04' ,'FKL33', 'FKL35', 'FKL36', 'FKL38', 'FKL39', 'FKL41', 'FKL42', 'FKL44', 'FKL45'], axis=1, inplace=True)

null_categorical_cols = ['FKL17', 'FKL17A', 'FKL18', 'FKL18A', 'FKL30']

for col in null_categorical_cols:
  value_counts = []
  col_value_counts = df[col].value_counts()
  for value, count in col_value_counts.items():
    value_counts.append(count)
  max_val = max(value_counts)
  for value, count in col_value_counts.items():
    if count == max_val:
      df[col].fillna(value, inplace=True)

df.isnull().sum()

/tmp/ipython-input-4038542596.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(value, inplace=True)
/tmp/ipython-input-4038542596.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.met

,0
Unnamed: 0,0
PSTV01,0
PSTV02,0
PSTV15,0
FKL05,0
FKL06,0
FKL07,0
FKL08,0
FKL09,0
FKL10,0


In [ ]:
duplicate_rows = df[df.duplicated()]
print("Number of duplicate rows:", duplicate_rows.shape[0])
display(duplicate_rows)

Number of duplicate rows: 0


,Unnamed: 0,PSTV01,PSTV02,PSTV15,FKL05,FKL06,FKL07,FKL08,FKL09,FKL10,...,FKL30,FKL31,FKL32,FKL34,FKL37,FKL40,FKL43,FKL46,FKL47,FKL48


## Label Distribution

In [ ]:
# Memeriksa distribusi label
value_counts = df['FKL23'].value_counts().items()
print('Distribusi kolom ' + col)
for key, value in value_counts:
  print(key + ': ' + str(value))


Distribusi kolom FKL30
 Rawat Jalan: 775621
 Ringan (rawat inap keparahan 1): 73694
 Sedang (rawat inap keparahan 2): 17807
 Berat (rawat inap keparahan 3): 5079


# Feature Engineering and Selection


In [ ]:
# Label encoding untuk kolom target
fkl23_label_col = np.array(df[['FKL23']].values)

label_col = fkl23_label_col.ravel()

le = preprocessing.LabelEncoder()

fkl23_labels_encoded = le.fit_transform(label_col)
print(fkl23_labels_encoded)

[2 2 2 ... 1 1 1]


In [ ]:
# Perbandingan dengan data sebelum diencoding
category_df = np.column_stack((label_col, fkl23_labels_encoded))
print(category_df)

[[' Ringan (rawat inap keparahan 1)' 2]
 [' Ringan (rawat inap keparahan 1)' 2]
 [' Ringan (rawat inap keparahan 1)' 2]
 ...
 [' Rawat Jalan' 1]
 [' Rawat Jalan' 1]
 [' Rawat Jalan' 1]]


In [ ]:
# Menampilkan setiap jenis label dan kodenya
unique_categories = []
for category in category_df:
  category_tuple = tuple(category)
  if category_tuple in unique_categories:
    pass
  else:
    unique_categories.append(category_tuple)

print(unique_categories)

[(' Ringan (rawat inap keparahan 1)', 2), (' Rawat Jalan', 1), (' Sedang (rawat inap keparahan 2)', 3), (' Berat (rawat inap keparahan 3)', 0)]


In [ ]:
# Melihat korelasi antara setiap fitur kategorikal dengan target
correlation_test_results = {}

# Perbarui list object_cols untuk hanya menyertakan kolom yang masih ada di df
object_cols = df.select_dtypes(include='object').columns

for col in object_cols:
  if col != 'FKL23':
    temp_df = df[[col, 'FKL23']].dropna()
    if not temp_df.empty:
        contingency_table = pd.crosstab(temp_df[col], temp_df['FKL23'])
        print("Hasil Uji Chi Squared & Cramer's untuk kolom " + col + ": ")
        print("Tabel Kontingensi kolom " + col + " dengan FKL23: ")
        display(contingency_table)

        chi2, p, dof, expected = chi2_contingency(contingency_table)

        print("\nHasil Uji Chi-Squared:")
        print(f"Chi-squared statistic: {chi2}")
        print(f"P-value: {p}")
        print(f"Degrees of freedom: {dof}")

        alpha = 0.05 # Tingkat signifikansi (umumnya 0.05)

        print("\nInterpretasi:")
        if p < alpha:
            print("P-value < alpha: Tolak hipotesis nol. Terdapat hubungan signifikan antara " + col + " dan FKL23.")
        else:
            print("P-value >= alpha: Gagal menolak hipotesis nol. Tidak terdapat bukti hubungan signifikan antara " + col + " dan FKL23 pada tingkat signifikansi ini.")

        from scipy.stats.contingency import association
        cramers_v = association(contingency_table, method='cramer')

        correlation_test_results[col] = cramers_v

        print(f"Cramer's V: {cramers_v}")
    else:
        print(f"Kolom {col} setelah dropna kosong, tidak bisa melakukan uji korelasi.")
    print('\n-----------------------------------------------------------------\n')

print("\nRingkasan Cramer's V:")
display(correlation_test_results)

Hasil Uji Chi Squared & Cramer's untuk kolom FKL05: 
Tabel Kontingensi kolom FKL05 dengan FKL23: 


FKL23,Berat (rawat inap keparahan 3),Rawat Jalan,Ringan (rawat inap keparahan 1),Sedang (rawat inap keparahan 2)
FKL05,,,,
ACEH,100,12986,2609,431
BALI,194,35283,2129,501
BANTEN,71,24336,2258,372
BENGKULU,6,6142,1215,69
DAERAH ISTIMEWA YOGYAKARTA,133,23059,1215,377
DKI JAKARTA,351,40081,2164,681
GORONTALO,51,2230,623,193
JAMBI,23,5161,1018,175
JAWA BARAT,693,111385,9614,2202



Hasil Uji Chi-Squared:
Chi-squared statistic: 12062.749231104975
P-value: 0.0
Degrees of freedom: 99

Interpretasi:
P-value < alpha: Tolak hipotesis nol. Terdapat hubungan signifikan antara FKL05 dan FKL23.
Cramer's V: 0.06789757011739939

-----------------------------------------------------------------

Hasil Uji Chi Squared & Cramer's untuk kolom FKL06: 
Tabel Kontingensi kolom FKL06 dengan FKL23: 


FKL23,Berat (rawat inap keparahan 3),Rawat Jalan,Ringan (rawat inap keparahan 1),Sedang (rawat inap keparahan 2)
FKL06,,,,
ACEH BARAT,5,408,139,8
ACEH BARAT DAYA,9,513,92,9
ACEH BESAR,3,293,62,4
ACEH JAYA,2,187,32,18
ACEH SELATAN,7,482,92,24
...,...,...,...,...
WAKATOBI,2,14,9,2
WAY KANAN,4,648,71,14
WONOGIRI,22,2392,200,72



Hasil Uji Chi-Squared:
Chi-squared statistic: 26795.82546065629
P-value: 0.0
Degrees of freedom: 1461

Interpretasi:
P-value < alpha: Tolak hipotesis nol. Terdapat hubungan signifikan antara FKL06 dan FKL23.
Cramer's V: 0.10119630437736478

-----------------------------------------------------------------

Hasil Uji Chi Squared & Cramer's untuk kolom FKL07: 
Tabel Kontingensi kolom FKL07 dengan FKL23: 


...

FKL23,Berat (rawat inap keparahan 3),Rawat Jalan,Ringan (rawat inap keparahan 1),Sedang (rawat inap keparahan 2)
FKL30,,,,
-,8,179,42,13
0002 - Therapeutic ultrasound of heart,0,3,0,0
0009 - Other therapeutic ultrasound,0,130,1,0
0009 - Other therapeutic ultrasound;0392 - Injection of other agent into spinal canal,0,0,1,0
0009 - Other therapeutic ultrasound;0449 - Other peripheral nerve or ganglion decompression or lysis of adhesions,0,1,0,0
...,...,...,...,...
9993 - Rectal massage (for levator spasm),0,1,0,0
9997 - Fitting of denture,0,23,0,0
9998 - Extraction of milk from lactating breast,0,1,0,0



Hasil Uji Chi-Squared:
Chi-squared statistic: 1341664.636232174
P-value: 0.0
Degrees of freedom: 98832

Interpretasi:
P-value < alpha: Tolak hipotesis nol. Terdapat hubungan signifikan antara FKL30 dan FKL23.
Cramer's V: 0.7160660258540745

-----------------------------------------------------------------

Hasil Uji Chi Squared & Cramer's untuk kolom FKL31: 
Tabel Kontingensi kolom FKL31 dengan FKL23: 


FKL23,Berat (rawat inap keparahan 3),Rawat Jalan,Ringan (rawat inap keparahan 1),Sedang (rawat inap keparahan 2)
FKL31,,,,
regional 1,2850,437294,33476,9023
regional 2,654,128781,11980,2539
regional 3,914,145841,20034,3762
regional 4,400,41810,4151,1288
regional 5,261,21895,4053,1195



Hasil Uji Chi-Squared:
Chi-squared statistic: 6590.016505794835
P-value: 0.0
Degrees of freedom: 12

Interpretasi:
P-value < alpha: Tolak hipotesis nol. Terdapat hubungan signifikan antara FKL31 dan FKL23.
Cramer's V: 0.05018504682299936

-----------------------------------------------------------------


Ringkasan Cramer's V:


{'FKL05': 0.06789757011739939,
 'FKL06': 0.10119630437736478,
 'FKL07': 0.038319947778208366,
 'FKL08': 0.04493108043702731,
 'FKL09': 0.09040057165586995,
 'FKL10': 1.0,
 'FKL11': 0.5773502691896258,
 'FKL12': 0.04701275009006912,
 'FKL13': 0.4579619924931029,
 'FKL14': 0.1554590015023317,
 'FKL15': 0.35881932440931263,
 'FKL15A': 0.35881932440931263,
 'FKL16': 0.3825769717253787,
 'FKL16A': 0.3825748810615786,
 'FKL17': 0.5022429749952517,
 'FKL17A': 0.5022429749952517,
 'FKL18': 0.5344299015766141,
 'FKL18A': 0.5344299015766141,
 'FKL19': 1.0,
 'FKL19A': 0.9994391066323232,
 'FKL20': 0.47444126329824077,
 'FKL21': 0.5852245439230346,
 'FKL25': 0.06993566442286013,
 'FKL26': 0.10857753018235794,
 'FKL27': 0.11863846057334271,
 'FKL28': 0.3275397582583922,
 'FKL29': 0.3463460911519932,
 'FKL30': 0.7160660258540745,
 'FKL31': 0.05018504682299936}

In [ ]:
# Memfilter hasil uji korelasi
selected_features = {}

for key, value in correlation_test_results.items():
  if value >= 0.3:
    selected_features[key] = value

print(selected_features)

{'FKL10': 1.0, 'FKL11': 0.5773502691896258, 'FKL13': 0.4579619924931029, 'FKL15': 0.35881932440931263, 'FKL15A': 0.35881932440931263, 'FKL16': 0.3825769717253787, 'FKL16A': 0.3825748810615786, 'FKL17': 0.5022429749952517, 'FKL17A': 0.5022429749952517, 'FKL18': 0.5344299015766141, 'FKL18A': 0.5344299015766141, 'FKL19': 1.0, 'FKL19A': 0.9994391066323232, 'FKL20': 0.47444126329824077, 'FKL21': 0.5852245439230346, 'FKL28': 0.3275397582583922, 'FKL29': 0.3463460911519932, 'FKL30': 0.7160660258540745}


In [ ]:
#Label Encoding
df_encoded = df.copy()
df_encoded = df[list(selected_features.keys())]

for col in selected_features:
  label_col = np.array(df[[col]].values)
  label_col = label_col.ravel()

  le = preprocessing.LabelEncoder()

  labels_encoded = le.fit_transform(label_col)

  df_encoded[col] = labels_encoded

df_encoded['FKL23'] = fkl23_labels_encoded
df_encoded


/tmp/ipython-input-2945298133.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encoded[col] = labels_encoded
/tmp/ipython-input-2945298133.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encoded[col] = labels_encoded
/tmp/ipython-input-2945298133.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

,FKL10,FKL11,FKL13,FKL15,FKL15A,FKL16,FKL16A,FKL17,FKL17A,FKL18,FKL18A,FKL19,FKL19A,FKL20,FKL21,FKL28,FKL29,FKL30,FKL23
0,0,27,2,1091,1090,5527,2253,1267,1267,5069,1596,653,339,15,5,7,5,5553,2
1,0,27,2,1266,1266,6121,2297,0,0,0,5132,10,414,0,3,7,23,5553,2
2,0,27,1,10,9,50,2652,0,0,0,5132,10,414,0,3,7,32,5553,2
3,0,27,1,2,1,5,7221,0,0,0,5132,10,414,0,3,7,28,5553,2
4,0,27,1,1266,1266,6121,2297,0,0,0,5132,10,414,0,3,7,27,31902,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872196,1,15,2,1580,1580,7369,2151,1339,1339,5366,1102,586,533,13,2,1,2,5553,1
872197,1,15,2,951,950,4824,1115,1339,1339,5366,1102,586,533,13,2,6,17,5553,1
872198,1,15,2,951,950,4826,1155,1339,1339,5366,1102,586,533,13,2,6,19,5553,1
872199,1,15,2,951,950,4824,1115,1339,1339,5366,1102,586,533,13,2,6,17,5553,1


In [ ]:
x = np.array(df_encoded[list(selected_features.keys())])
y = np.array(df_encoded[['FKL23']]).ravel()

In [ ]:
# Undersampling
print("Distribusi kelas sebelum undersampling:")
print(Counter(y))

rus = RandomUnderSampler(random_state=0, sampling_strategy={0: 2000, 1: 2000, 2: 2000, 3: 2000})

x_resampled, y_resampled = rus.fit_resample(x, y)

print("\nDistribusi kelas setelah undersampling:")
print(Counter(y_resampled))
print("\nUkuran data setelah undersampling:")
print("x_resampled shape:", x_resampled.shape)
print("y_resampled shape:", y_resampled.shape)

df_resampled = pd.DataFrame(x_resampled, columns=list(selected_features.keys()))
df_resampled['FKL23'] = y_resampled

display(df_resampled.head())

Distribusi kelas sebelum undersampling:
Counter({np.int64(1): 775621, np.int64(2): 73694, np.int64(3): 17807, np.int64(0): 5079})

Distribusi kelas setelah undersampling:
Counter({np.int64(0): 2000, np.int64(1): 2000, np.int64(2): 2000, np.int64(3): 2000})

Ukuran data setelah undersampling:
x_resampled shape: (8000, 18)
y_resampled shape: (8000,)


,FKL10,FKL11,FKL13,FKL15,FKL15A,FKL16,FKL16A,FKL17,FKL17A,FKL18,FKL18A,FKL19,FKL19A,FKL20,FKL21,FKL28,FKL29,FKL30,FKL23
0,0,27,2,323,322,1249,4272,265,265,776,2219,489,566,12,1,7,21,16440,0
1,0,27,2,1227,1227,5989,1976,472,472,1514,2005,589,818,13,3,7,21,5993,0
2,0,27,2,253,252,921,4665,135,135,321,2471,350,833,9,3,7,20,4035,0
3,0,27,1,149,148,482,3715,243,243,716,245,91,155,3,3,7,21,32748,0
4,0,27,2,786,785,3319,2757,7,7,27,1836,433,363,10,3,7,22,5553,0


In [ ]:
# Kombinasi atribut 1 (nilai Cramer's >= 0.7)
x2 = np.array(df_resampled[['FKL10', 'FKL19', 'FKL19A', 'FKL30']])
y2 = np.array(df_resampled[['FKL23']]).ravel()

# Kombinasi atribut 2 (nilai Cramer's >= 0.5)
x3 = np.array(df_resampled[['FKL21', 'FKL18A', 'FKL18', 'FKL17A', 'FKL17', 'FKL11']])
y3 = np.array(df_resampled[['FKL23']]).ravel()

# Train and Evaluation

## Random Forest

### Kombinasi atribut 1


In [ ]:
# Train test split
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(x2, y2, test_size=0.3, random_state=0)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(x3, y3, test_size=0.3, random_state=0)
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(x_resampled, y_resampled, test_size=0.3, random_state=0)

In [ ]:
%%time
# Melatih Random Forest 1 dengan Bayesian Optimization
# Fungsi Objektif
def rf_objective_function(max_features, n_estimators, max_depth, min_samples_split, min_samples_leaf):
    n_estimators = int(n_estimators) if n_estimators > 0 else 1

    max_depth_int = int(max_depth) if max_depth is not None and max_depth >= 1 else None

    max_features_int = int(max_features) if max_features > 0 else 1

    min_samples_split_int = int(min_samples_split) if min_samples_split > 0 else 1

    min_samples_leaf_int = int(min_samples_leaf) if min_samples_leaf > 0 else 1

    model = RandomForestClassifier(
        max_features=max_features_int,
        n_estimators=n_estimators,
        max_depth=max_depth_int,
        min_samples_leaf=min_samples_leaf_int,
        min_samples_split=min_samples_split_int,
        random_state=0,
        n_jobs=-1
    )

    scores = cross_val_score(model, X_train_1, y_train_1, cv=4, scoring='accuracy', n_jobs=-1)
    return scores.mean()

# Rentang Nilai Parameter
pbounds={
    'max_features': (1.0, 10.0),
    'n_estimators': (50.0, 300.0),
    'max_depth': (5.0, 50.0),
    'min_samples_split': (2.0, 50.0),
    'min_samples_leaf': (1.0, 50.0)
}

# Membuat Optimizer
optimizer= BayesianOptimization(
    f=rf_objective_function,
    pbounds=pbounds,
    random_state=0
)

# Melatih Optimizer
optimizer.maximize(
    init_points=5,
    n_iter=20,
)

# Menampilkan Hasil Optimisasi Terbaik
print('Hasil terbaik: ', optimizer.max)

best_params = optimizer.max['params']

# Melatih Model Random Forest Menggnakan Hasil Optimisasi Terbaik
best_rf_model_bo = RandomForestClassifier(
    n_estimators=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']) if best_params['max_depth'] is not None and best_params['max_depth'] >= 1 else None, # Convert back to int/None with validation
    max_features=int(best_params['max_features']) if best_params['max_features'] > 0 else 1, # Convert back to int with validation
    random_state=0,
    n_jobs=-1
)

best_rf_model_bo.fit(X_train_1, y_train_1)

# Mengevaluasi Model
y_pred_bo = best_rf_model_bo.predict(X_test_1)

acc_bo = accuracy_score(y_test_1, y_pred_bo)
print(f'\nAccuracy of best Bayesian Optimization model on test set: {acc_bo:.4f}')
print(f'Classification report:\n{classification_report(y_test_1, y_pred_bo)}')
print(f'Confusion matrix:\n{confusion_matrix(y_test_1, y_pred_bo)}')

|   iter    |  target   | max_fe... | n_esti... | max_depth | min_sa... | min_sa... |
-------------------------------------------------------------------------------------
| 1         | 0.8017857 | 5.9393215 | 228.79734 | 32.124351 | 28.154392 | 21.759085 |
| 2         | 0.8042857 | 6.8130470 | 159.39680 | 45.129785 | 48.255812 | 19.788634 |
| 3         | 0.8674999 | 8.1255253 | 182.22372 | 30.562005 | 46.428638 | 4.4807668 |
| 4         | 0.7078571 | 1.7841636 | 55.054599 | 42.467893 | 39.351524 | 43.630595 |
| 5         | 0.86625   | 9.8075650 | 249.78964 | 25.766571 | 39.465400 | 6.7954468 |
| 6         | 0.8626785 | 9.9995312 | 250.42914 | 26.259701 | 40.155023 | 7.5747479 |
| 7         | 0.6823214 | 8.2966745 | 258.73579 | 5.0       | 26.534491 | 1.0       |
| 8         | 0.8692857 | 9.2815796 | 237.62056 | 32.030869 | 44.948776 | 1.0       |
| 9         | 0.8894642 | 9.5207771 | 247.54515 | 42.901611 | 30.934656 | 1.0       |
| 10        | 0.8076785 | 1.0       | 254.76996 | 50.0

### Kombinasi Atribut 2

In [ ]:
%%time
# Melatih Random Forest 2 dengan Bayesian Optimization
# Fungsi Objektif
def rf_objective_function_2(max_features, n_estimators, max_depth, min_samples_split, min_samples_leaf):
    n_estimators = int(n_estimators) if n_estimators > 0 else 1

    max_depth_int = int(max_depth) if max_depth is not None and max_depth >= 1 else None

    max_features_int = int(max_features) if max_features > 0 else 1

    min_samples_split_int = int(min_samples_split) if min_samples_split > 0 else 1

    min_samples_leaf_int = int(min_samples_leaf) if min_samples_leaf > 0 else 1

    model = RandomForestClassifier(
        max_features=max_features_int,
        n_estimators=n_estimators,
        max_depth=max_depth_int,
        min_samples_leaf=min_samples_leaf_int,
        min_samples_split=min_samples_split_int,
        random_state=0,
        n_jobs=-1
    )

    scores = cross_val_score(model, X_train_2, y_train_2, cv=4, scoring='accuracy', n_jobs=-1)
    return scores.mean()

# Rentang Nilai Parameter
pbounds_2={
    'max_features': (1.0, 10.0),
    'n_estimators': (50.0, 300.0),
    'max_depth': (5.0, 50.0),
    'min_samples_split': (2.0, 50.0),
    'min_samples_leaf': (1.0, 50.0)
}

# Membuat Optimizer
optimizer_2 = BayesianOptimization(
    f=rf_objective_function_2,
    pbounds=pbounds_2,
    random_state=0
)

# Melatih Optimizer
optimizer_2.maximize(
    init_points=5,
    n_iter=20,
)

# Menampilkan Hasil Optimisasi Terbaik
print('Hasil terbaik: ', optimizer_2.max)

best_params_2 = optimizer_2.max['params']

# Melatih Model Random Forest Menggnakan Hasil Optimisasi Terbaik
best_rf_model_bo_2 = RandomForestClassifier(
    n_estimators=int(best_params_2['n_estimators']),
    max_depth=int(best_params_2['max_depth']) if best_params_2['max_depth'] is not None and best_params_2['max_depth'] >= 1 else None,
    max_features=int(best_params_2['max_features']) if best_params_2['max_features'] > 0 else 1,
    random_state=0,
    n_jobs=-1
)

best_rf_model_bo_2.fit(X_train_2, y_train_2)

# Mengevaluasi Model
y_pred_bo_2 = best_rf_model_bo_2.predict(X_test_2)

acc_bo_2 = accuracy_score(y_test_2, y_pred_bo_2)
print(f'\nAccuracy of best Bayesian Optimization model on test set: {acc_bo_2:.4f}')
print(f'Classification report:\n{classification_report(y_test_2, y_pred_bo_2)}')
print(f'Confusion matrix:\n{confusion_matrix(y_test_2, y_pred_bo_2)}')

|   iter    |  target   | max_fe... | n_esti... | max_depth | min_sa... | min_sa... |
-------------------------------------------------------------------------------------
| 1         | 0.6448214 | 5.9393215 | 228.79734 | 32.124351 | 28.154392 | 21.759085 |
| 2         | 0.6425    | 6.8130470 | 159.39680 | 45.129785 | 48.255812 | 19.788634 |
| 3         | 0.6483928 | 8.1255253 | 182.22372 | 30.562005 | 46.428638 | 4.4807668 |
| 4         | 0.6160714 | 1.7841636 | 55.054599 | 42.467893 | 39.351524 | 43.630595 |
| 5         | 0.6473214 | 9.8075650 | 249.78964 | 25.766571 | 39.465400 | 6.7954468 |
| 6         | 0.6010714 | 1.0       | 300.0     | 5.0       | 50.0      | 50.0      |
| 7         | 0.6475    | 9.7670715 | 249.83243 | 25.353991 | 40.785207 | 7.3545418 |
| 8         | 0.6091071 | 10.0      | 158.76291 | 5.0       | 2.0       | 1.0       |
| 9         | 0.6503571 | 10.0      | 219.82524 | 50.0      | 50.0      | 1.0       |
| 10        | 0.6094642 | 10.0      | 194.31079 | 5.0 

### Kombinasi Atribut 3

In [ ]:
%%time
# Melatih Random Forest 3 dengan Bayesian Optimization
# Fungsi Objektif
def rf_objective_function_3(max_features, n_estimators, max_depth, min_samples_split, min_samples_leaf):
    n_estimators = int(n_estimators) if n_estimators > 0 else 1

    max_depth_int = int(max_depth) if max_depth is not None and max_depth >= 1 else None

    max_features_int = int(max_features) if max_features > 0 else 1

    min_samples_split_int = int(min_samples_split) if min_samples_split > 0 else 1

    min_samples_leaf_int = int(min_samples_leaf) if min_samples_leaf > 0 else 1

    model = RandomForestClassifier(
        max_features=max_features_int,
        n_estimators=n_estimators,
        max_depth=max_depth_int,
        min_samples_leaf=min_samples_leaf_int,
        min_samples_split=min_samples_split_int,
        random_state=0,
        n_jobs=-1
    )

    scores = cross_val_score(model, X_train_3, y_train_3, cv=4, scoring='accuracy', n_jobs=-1)
    return scores.mean()

# Rentang Nilai Parameter
pbounds_3={
    'max_features': (1.0, 10.0),
    'n_estimators': (50.0, 300.0),
    'max_depth': (5.0, 50.0),
    'min_samples_split': (2.0, 50.0),
    'min_samples_leaf': (1.0, 50.0)
}

# Membuat Optimizer
optimizer_3 = BayesianOptimization(
    f=rf_objective_function_3,
    pbounds=pbounds_3,
    random_state=0
)

# Melatih Optimizer
optimizer_3.maximize(
    init_points=5,
    n_iter=20,
)

# Menampilkan Hasil Optimisasi Terbaik
print('Hasil terbaik: ', optimizer_3.max)

best_params_3 = optimizer_3.max['params']

# Melatih Model Random Forest Menggnakan Hasil Optimisasi Terbaik
best_rf_model_bo_3 = RandomForestClassifier(
    n_estimators=int(best_params_3['n_estimators']),
    max_depth=int(best_params_3['max_depth']) if best_params_3['max_depth'] is not None and best_params_3['max_depth'] >= 1 else None,
    max_features=int(best_params_3['max_features']) if best_params_3['max_features'] > 0 else 1,
    random_state=0,
    n_jobs=-1
)

best_rf_model_bo_3.fit(X_train_3, y_train_3)

# Mengevaluasi Model
y_pred_bo_3 = best_rf_model_bo_3.predict(X_test_3)

acc_bo_3 = accuracy_score(y_test_3, y_pred_bo_3)
print(f'\nAccuracy of best Bayesian Optimization model on test set: {acc_bo_3:.4f}')
print(f'Classification report:\n{classification_report(y_test_3, y_pred_bo_3)}')
print(f'Confusion matrix:\n{confusion_matrix(y_test_3, y_pred_bo_3)}')

|   iter    |  target   | max_fe... | n_esti... | max_depth | min_sa... | min_sa... |
-------------------------------------------------------------------------------------
| 1         | 0.7448214 | 5.9393215 | 228.79734 | 32.124351 | 28.154392 | 21.759085 |
| 2         | 0.7548214 | 6.8130470 | 159.39680 | 45.129785 | 48.255812 | 19.788634 |
| 3         | 0.8044642 | 8.1255253 | 182.22372 | 30.562005 | 46.428638 | 4.4807668 |
| 4         | 0.66      | 1.7841636 | 55.054599 | 42.467893 | 39.351524 | 43.630595 |
| 5         | 0.8133928 | 9.8075650 | 249.78964 | 25.766571 | 39.465400 | 6.7954468 |
| 6         | 0.8107142 | 9.7952616 | 250.29307 | 25.789279 | 40.132855 | 7.7558830 |
| 7         | 0.7546428 | 10.0      | 263.43889 | 7.0722341 | 22.161660 | 1.0       |
| 8         | 0.8282142 | 10.0      | 256.22765 | 46.206868 | 35.368069 | 1.0       |
| 9         | 0.7105357 | 1.0       | 281.06943 | 50.0      | 50.0      | 1.0       |
| 10        | 0.8178571 | 10.0      | 239.61814 | 44.8

## Support Vector Machine (SVM)

### Kombinasi atribut 1

In [ ]:
%%time
# Melatih SVM 1 dengan bayesian optimization
# Membuat fungsi objektif
def svm_objective_function(C, gamma):
  C = max(C, 1e-6)
  gamma = max(gamma, 1e-6)

  model = SVC(C=C, gamma=gamma, kernel='rbf', random_state=0)

  scores = cross_val_score(model, X_train_1, y_train_1, scoring='accuracy', n_jobs=-1)

  return scores.mean()

# Rentang nilai parameter
pbounds_svm_1 = {
    'C': (0.1, 100),
    'gamma': (0.001, 10)
}

# Membuat optimizer
optimizer_svm_1 = BayesianOptimization(
    f=svm_objective_function,
    pbounds=pbounds_svm_1,
    random_state=0
)

# Melatih optimizer
optimizer_svm_1.maximize(
    init_points=5,
    n_iter=20
)

# Menampilkan hasil terbaik
print('Hasil terbaik: ', optimizer_svm_1.max)

best_params_svm = optimizer_svm_1.max['params']

# Melatih model SVM menggunakan parameter terbaik
svm = SVC(
    C=best_params_svm['C'],
    gamma=best_params_svm['gamma'],
    random_state=0,
    kernel='rbf'
)

svm.fit(X_train_1, y_train_1)

# Mengevaluasi model
y_pred_svm = svm.predict(X_test_1)

acc_svm_bo = accuracy_score(y_test_1, y_pred_svm)
print(f'\nAccuracy of best Bayesian Optimization SVM model on test set: {acc_svm_bo:.4f}')
print(f'Classification report:\n{classification_report(y_test_1, y_pred_svm)}')
print(f'Confusion Matrix:\n{confusion_matrix(y_test_1, y_pred_svm)}')

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
| 1         | 0.6621428 | 54.926469 | 7.1521784 |
| 2         | 0.6621428 | 60.316061 | 5.4492869 |
| 3         | 0.6621428 | 42.423114 | 6.4592952 |
| 4         | 0.6621428 | 43.814962 | 8.9178382 |
| 5         | 0.6625    | 96.369909 | 3.8350317 |
| 6         | 0.6621428 | 99.802476 | 9.7096902 |
| 7         | 0.6746428 | 94.840640 | 1.4340511 |
| 8         | 0.6621428 | 30.351200 | 4.5610220 |
| 9         | 0.6621428 | 28.964957 | 5.4578462 |
| 10        | 0.6776785 | 94.475515 | 0.8349647 |
| 11        | 0.6867857 | 92.303364 | 0.001     |
| 12        | 0.6853571 | 87.456157 | 0.001     |
| 13        | 0.6621428 | 85.676339 | 6.5754962 |
| 14        | 0.6855357 | 89.933228 | 0.001     |
| 15        | 0.6823214 | 81.016478 | 0.001     |
| 16        | 0.6816071 | 73.517005 | 0.001     |
| 17        | 0.6621428 | 73.136553 | 7.0766828 |
| 18        | 0.5719642 | 0.2496133 | 0.2027930 |


### Kombinasi atribut 2

In [ ]:
%%time
# Melatih SVM 2 dengan bayesian optimization
# Membuat fungsi objektif
def svm_objective_function_2(C, gamma):
  C = max(C, 1e-6)
  gamma = max(gamma, 1e-6)

  model = SVC(C=C, gamma=gamma, kernel='rbf', random_state=0)

  scores = cross_val_score(model, X_train_2, y_train_2, scoring='accuracy', n_jobs=-1)

  return scores.mean()

# Rentang nilai parameter
pbounds_svm_2 = {
    'C': (0.1, 100),
    'gamma': (0.001, 10)
}

# Membuat optimizer
optimizer_svm_2 = BayesianOptimization(
    f=svm_objective_function_2,
    pbounds=pbounds_svm_2,
    random_state=0
)

# Melatih optimizer
optimizer_svm_2.maximize(
    init_points=5,
    n_iter=20
)

# Menampilkan hasil terbaik
print('Hasil terbaik: ', optimizer_svm_2.max)

best_params_svm_2 = optimizer_svm_2.max['params']

# Melatih model SVM menggunakan parameter terbaik
svm_2 = SVC(
    C=best_params_svm_2['C'],
    gamma=best_params_svm_2['gamma'],
    random_state=0,
    kernel='rbf'
)

svm_2.fit(X_train_2, y_train_2)

# Mengevaluasi model
y_pred_svm_2 = svm_2.predict(X_test_2)

acc_svm_bo_2 = accuracy_score(y_test_2, y_pred_svm_2)
print(f'\nAccuracy of best Bayesian Optimization SVM model on test set: {acc_svm_bo_2:.4f}')
print(f'Classification report:\n{classification_report(y_test_2, y_pred_svm_2)}')
print(f'Confusion Matrix:\n{confusion_matrix(y_test_2, y_pred_svm_2)}')

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
| 1         | 0.5717857 | 54.926469 | 7.1521784 |
| 2         | 0.5717857 | 60.316061 | 5.4492869 |
| 3         | 0.5717857 | 42.423114 | 6.4592952 |
| 4         | 0.5717857 | 43.814962 | 8.9178382 |
| 5         | 0.5716071 | 96.369909 | 3.8350317 |
| 6         | 0.5491071 | 0.2467537 | 9.9760298 |
| 7         | 0.5717857 | 81.356400 | 10.0      |
| 8         | 0.6108928 | 80.948497 | 0.0010000 |
| 9         | 0.6105357 | 75.686052 | 0.001     |
| 10        | 0.6092857 | 23.155170 | 0.001     |
| 11        | 0.5717857 | 23.841609 | 10.0      |
| 12        | 0.6074999 | 16.009743 | 0.001     |
| 13        | 0.6094642 | 30.657092 | 0.001     |
| 14        | 0.6073214 | 8.8564329 | 0.001     |
| 15        | 0.6101785 | 69.562320 | 0.001     |
| 16        | 0.6105357 | 86.873508 | 0.001     |
| 17        | 0.57125   | 71.646582 | 4.7670384 |
| 18        | 0.5717857 | 12.771893 | 5.7915499 |


### Kombinasi atribut 3

In [ ]:
%%time
# Melatih SVM 3 dengan bayesian optimization
# Membuat fungsi objektif
def svm_objective_function_3(C, gamma):
  C = max(C, 1e-6)
  gamma = max(gamma, 1e-6)

  model = SVC(C=C, gamma=gamma, kernel='rbf', random_state=0)

  scores = cross_val_score(model, X_train_3, y_train_3, scoring='accuracy', n_jobs=-1)

  return scores.mean()

# Rentang nilai parameter
pbounds_svm_3 = {
    'C': (0.1, 100),
    'gamma': (0.001, 10)
}

# Membuat optimizer
optimizer_svm_3 = BayesianOptimization(
    f=svm_objective_function_3,
    pbounds=pbounds_svm_3,
    random_state=0
)

# Melatih optimizer
optimizer_svm_3.maximize(
    init_points=5,
    n_iter=20
)

# Menampilkan hasil terbaik
print('Hasil terbaik: ', optimizer_svm_3.max)

best_params_svm_3 = optimizer_svm_3.max['params']

# Melatih model SVM menggunakan parameter terbaik
svm_3 = SVC(
    C=best_params_svm_3['C'],
    gamma=best_params_svm_3['gamma'],
    random_state=0,
    kernel='rbf'
)

svm_3.fit(X_train_3, y_train_3)

# Mengevaluasi model
y_pred_svm_3 = svm_3.predict(X_test_3)

acc_svm_bo_3 = accuracy_score(y_test_3, y_pred_svm_3)
print(f'\nAccuracy of best Bayesian Optimization SVM model on test set: {acc_svm_bo_3:.4f}')
print(f'Classification report:\n{classification_report(y_test_3, y_pred_svm_3)}')
print(f'Confusion Matrix:\n{confusion_matrix(y_test_3, y_pred_svm_3)}')

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
| 1         | 0.30125   | 54.926469 | 7.1521784 |
| 2         | 0.3039285 | 60.316061 | 5.4492869 |
| 3         | 0.3025    | 42.423114 | 6.4592952 |
| 4         | 0.30125   | 43.814962 | 8.9178382 |
| 5         | 0.3078571 | 96.369909 | 3.8350317 |
| 6         | 0.3099999 | 89.466909 | 3.2372457 |
| 7         | 0.30125   | 80.324247 | 10.0      |
| 8         | 0.4180357 | 91.726352 | 0.001     |
| 9         | 0.4180357 | 92.571661 | 0.001     |
| 10        | 0.2892857 | 0.1755753 | 0.0131368 |
| 11        | 0.4180357 | 95.291785 | 0.001     |
| 12        | 0.4180357 | 99.976413 | 0.001     |
| 13        | 0.30125   | 20.433336 | 9.9503168 |
| 14        | 0.4155357 | 30.073460 | 0.001     |
| 15        | 0.4150000 | 24.808573 | 0.001     |
| 16        | 0.3039285 | 29.423604 | 5.4119814 |
| 17        | 0.4150000 | 18.707260 | 0.001     |
| 18        | 0.4141071 | 12.671013 | 0.001     |


## K-Nearest Neighbor (KNN)

### Kombinasi atribut 1

In [ ]:
%%time
# Melatih KNN 1 dengan bayesian optimization
def knn_objective_function(n_neighbors, p):
    n_neighbors = int(round(n_neighbors))
    n_neighbors = max(1, n_neighbors)

    p = int(round(p))
    p = max(1, p)

    model = KNeighborsClassifier(n_neighbors=n_neighbors, p=p, weights='uniform', n_jobs=-1)

    scores = cross_val_score(model, X_train_1, y_train_1, cv=4, scoring='accuracy', n_jobs=-1)
    return scores.mean()

pbounds_knn = {
    'n_neighbors': (1, 30),
    'p': (1, 2)
}

# Membuat optimizer
optimizer_knn = BayesianOptimization(
    f=knn_objective_function,
    pbounds=pbounds_knn,
    random_state=0
)

# Melatih optimizer
optimizer_knn.maximize(
    init_points=5,
    n_iter=20,
)

# Menampilkan hasil optimisasi terbaik
print('Hasil terbaik: ', optimizer_knn.max)

# Mendapatkan parameter terbaik
best_params_knn = optimizer_knn.max['params']

# Melatih model KNN menggunakan hasil optimisasi terbaik
best_knn_model_bo = KNeighborsClassifier(
    n_neighbors=int(round(best_params_knn['n_neighbors'])),
    p=int(round(best_params_knn['p'])),
    weights='uniform',
    n_jobs=-1
)

best_knn_model_bo.fit(X_train_1, y_train_1)

# Mengevaluasi model terbaik pada data test
y_pred_knn_bo = best_knn_model_bo.predict(X_test_1)

acc_knn_bo = accuracy_score(y_test_1, y_pred_knn_bo)
print(f'\nAccuracy of best Bayesian Optimization KNN model on test set: {acc_knn_bo:.4f}')
print(f'Classification report:\n{classification_report(y_test_1, y_pred_knn_bo)}')
print(f'Confusion matrix:\n{confusion_matrix(y_test_1, y_pred_knn_bo)}')

|   iter    |  target   | n_neig... |     p     |
-------------------------------------------------
| 1         | 0.6342857 | 16.915591 | 1.7151893 |
| 2         | 0.6285714 | 18.480137 | 1.5448831 |
| 3         | 0.6385714 | 13.285989 | 1.6458941 |
| 4         | 0.6398214 | 13.690029 | 1.8917730 |
| 5         | 0.6146428 | 28.946220 | 1.3834415 |
| 6         | 0.7146428 | 1.0       | 2.0       |
| 7         | 0.6792857 | 3.5922042 | 2.0       |
| 8         | 0.7269642 | 1.0       | 1.0       |
| 9         | 0.7269642 | 1.0       | 1.0       |
| 10        | 0.7269642 | 1.0       | 1.0       |
| 11        | 0.7269642 | 1.0       | 1.0       |
| 12        | 0.7269642 | 1.0       | 1.0       |
| 13        | 0.7269642 | 1.0       | 1.0       |
| 14        | 0.7269642 | 1.0       | 1.0       |
| 15        | 0.7269642 | 1.0       | 1.0       |
| 16        | 0.7269642 | 1.0       | 1.0       |
| 17        | 0.7269642 | 1.0       | 1.0       |
| 18        | 0.7269642 | 1.0       | 1.0       |


### Kombinasi atribut 2

In [ ]:
%%time
# Melatih KNN 2 dengan bayesian optimization
def knn_objective_function_2(n_neighbors, p):
    n_neighbors = int(round(n_neighbors))
    n_neighbors = max(1, n_neighbors)

    p = int(round(p))
    p = max(1, p)

    model = KNeighborsClassifier(n_neighbors=n_neighbors, p=p, weights='uniform', n_jobs=-1)

    scores = cross_val_score(model, X_train_2, y_train_2, cv=4, scoring='accuracy', n_jobs=-1)
    return scores.mean()

pbounds_knn_2 = {
    'n_neighbors': (1, 30),
    'p': (1, 2)
}

# Membuat optimizer
optimizer_knn_2 = BayesianOptimization(
    f=knn_objective_function_2,
    pbounds=pbounds_knn_2,
    random_state=0
)

# Melatih optimizer
optimizer_knn_2.maximize(
    init_points=5,
    n_iter=20,
)

# Menampilkan hasil optimisasi terbaik
print('Hasil terbaik: ', optimizer_knn_2.max)

# Mendapatkan parameter terbaik
best_params_knn_2 = optimizer_knn_2.max['params']

# Melatih model KNN menggunakan hasil optimisasi terbaik
best_knn_model_bo_2 = KNeighborsClassifier(
    n_neighbors=int(round(best_params_knn_2['n_neighbors'])),
    p=int(round(best_params_knn_2['p'])),
    weights='uniform',
    n_jobs=-1
)

best_knn_model_bo_2.fit(X_train_2, y_train_2)

# Mengevaluasi model terbaik pada data test
y_pred_knn_bo_2 = best_knn_model_bo_2.predict(X_test_2)

acc_knn_bo_2 = accuracy_score(y_test_2, y_pred_knn_bo_2)
print(f'\nAccuracy of best Bayesian Optimization KNN model on test set: {acc_knn_bo_2:.4f}')
print(f'Classification report:\n{classification_report(y_test_2, y_pred_knn_bo_2)}')
print(f'Confusion matrix:\n{confusion_matrix(y_test_2, y_pred_knn_bo_2)}')

|   iter    |  target   | n_neig... |     p     |
-------------------------------------------------
| 1         | 0.5830357 | 16.915591 | 1.7151893 |
| 2         | 0.5853571 | 18.480137 | 1.5448831 |
| 3         | 0.5883928 | 13.285989 | 1.6458941 |
| 4         | 0.5858928 | 13.690029 | 1.8917730 |
| 5         | 0.58375   | 28.946220 | 1.3834415 |
| 6         | 0.5883928 | 13.268621 | 1.6532753 |
| 7         | 0.5908928 | 13.053747 | 1.0331292 |
| 8         | 0.59      | 11.838978 | 1.0       |
| 9         | 0.5892857 | 9.7670981 | 1.0       |
| 10        | 0.5880357 | 7.8783421 | 1.9684137 |
| 11        | 0.59625   | 4.9945675 | 1.0082105 |
| 12        | 0.5923214 | 3.6938461 | 1.0       |
| 13        | 0.5930357 | 5.1186174 | 2.0       |
| 14        | 0.5953571 | 5.9903442 | 1.0       |
| 15        | 0.5842857 | 23.231696 | 1.0197429 |
| 16        | 0.5733928 | 1.0       | 2.0       |
| 17        | 0.59625   | 5.4113074 | 1.0       |
| 18        | 0.59625   | 5.2171796 | 1.1280320 |


### Kombinasi atribut 3

In [ ]:
%%time
# Melatih KNN 3 dengan bayesian optimization
def knn_objective_function_3(n_neighbors, p):
    n_neighbors = int(round(n_neighbors))
    n_neighbors = max(1, n_neighbors)

    p = int(round(p))
    p = max(1, p)

    model = KNeighborsClassifier(n_neighbors=n_neighbors, p=p, weights='uniform', n_jobs=-1)

    scores = cross_val_score(model, X_train_3, y_train_3, cv=4, scoring='accuracy', n_jobs=-1)
    return scores.mean()

pbounds_knn_3 = {
    'n_neighbors': (1, 30),
    'p': (1, 2)
}

# Membuat optimizer
optimizer_knn_3 = BayesianOptimization(
    f=knn_objective_function_3,
    pbounds=pbounds_knn_3,
    random_state=0
)

# Melatih optimizer
optimizer_knn_3.maximize(
    init_points=5,
    n_iter=20,
)

# Menampilkan hasil optimisasi terbaik
print('Hasil terbaik: ', optimizer_knn_3.max)

# Mendapatkan parameter terbaik
best_params_knn_3 = optimizer_knn_3.max['params']

# Melatih model KNN menggunakan hasil optimisasi terbaik
best_knn_model_bo_3 = KNeighborsClassifier(
    n_neighbors=int(round(best_params_knn_3['n_neighbors'])),
    p=int(round(best_params_knn_3['p'])),
    weights='uniform',
    n_jobs=-1
)

best_knn_model_bo_3.fit(X_train_3, y_train_3)

# Mengevaluasi model terbaik pada data test
y_pred_knn_bo_3 = best_knn_model_bo_3.predict(X_test_3)

acc_knn_bo_3 = accuracy_score(y_test_3, y_pred_knn_bo_3)
print(f'\nAccuracy of best Bayesian Optimization KNN model on test set: {acc_knn_bo_3:.4f}')
print(f'Classification report:\n{classification_report(y_test_3, y_pred_knn_bo_3)}')
print(f'Confusion matrix:\n{confusion_matrix(y_test_3, y_pred_knn_bo_3)}')

|   iter    |  target   | n_neig... |     p     |
-------------------------------------------------
| 1         | 0.5442857 | 16.915591 | 1.7151893 |
| 2         | 0.54375   | 18.480137 | 1.5448831 |
| 3         | 0.5439285 | 13.285989 | 1.6458941 |
| 4         | 0.54625   | 13.690029 | 1.8917730 |
| 5         | 0.5694642 | 28.946220 | 1.3834415 |
| 6         | 0.5367857 | 26.454394 | 2.0       |
| 7         | 0.5719642 | 29.831214 | 1.1630266 |
| 8         | 0.5373214 | 30.0      | 2.0       |
| 9         | 0.5719642 | 29.816643 | 1.1613402 |
| 10        | 0.5694642 | 28.345818 | 1.0       |
| 11        | 0.5358928 | 28.183086 | 1.9086794 |
| 12        | 0.5694642 | 29.031930 | 1.0       |
| 13        | 0.5746428 | 1.0722750 | 1.4009673 |
| 14        | 0.5544642 | 1.8331345 | 1.6565357 |
| 15        | 0.5508928 | 1.0       | 2.0       |
| 16        | 0.5746428 | 1.0       | 1.0       |
| 17        | 0.5739285 | 1.5279199 | 1.0       |
| 18        | 0.5864285 | 7.4636418 | 1.0590809 |


# Testing Best Model

In [ ]:
# Menyimpan model ke format .pkl
best_model = best_rf_model_bo

filename = '/content/drive/MyDrive/Machine Learning Models/Supervised Learning/Classification/Random Forest/best_random_forest_model.pkl'

with open(filename, 'wb') as file:
  pickle.dump(best_model, file)

print(f"Model berhasil disimpan ke dalam file '{filename}'")

Model berhasil disimpan ke dalam file '/content/drive/MyDrive/Machine Learning Models/Supervised Learning/Classification/Random Forest/best_random_forest_model.pkl'


In [ ]:

# Menampilkan data baru
df_new = pd.read_csv('/content/drive/MyDrive/Machine Learning Datasets/UTS/synthetic_data_subset.csv')
df_new.head()

,FKL10,FKL19,FKL19A,FKL30
0,RJTL,S-4-16-III,DIAGNOSIS SISTEM MUSKULOSKLETAL & JARINGAN PE...,1621 - Ophthalmoscopy;8911 - Tonometry;9511 - ...
1,RJTL,W-1-30-III,GANGGUAN JARINGAN KONEKTIF BERAT,9904 - Transfusion of packed cells;9923 - Inje...
2,RJTL,M-1-02-I,KECEDERAAN PEMBULUH DARAH OTAK NON SPESIFIK &...,9327 - Stretching of muscle or tendon;9339 - O...
3,RJTL,A-4-14-I,PROSEDUR KECIL PADA MATA,8950 - Ambulatory cardiac monitoring;9059 - Ot...
4,RITL,J-4-21-III,PROSEDUR KAKI SEDANG,9916 - Injection of antidote;9929 - Injection ...


In [ ]:
# Encoding data baru
df_new_encoded = pd.DataFrame()

for col in df_new.columns:
  feature_col = np.array(df_new[[col]].values)
  feature_col = feature_col.ravel()

  le = preprocessing.LabelEncoder()

  features_encoded = le.fit_transform(feature_col)

  df_new_encoded[col] = features_encoded

df_new_encoded

,FKL10,FKL19,FKL19A,FKL30
0,1,68,9,7
1,1,82,18,97
2,1,47,38,94
3,1,0,74,85
4,0,35,70,98
...,...,...,...,...
95,0,16,62,2
96,0,40,65,83
97,1,10,5,29
98,0,59,97,75


In [ ]:
# Memuat model random forest dari file pickle dan membuat prediksi
filename = '/content/drive/MyDrive/Machine Learning Models/Supervised Learning/Classification/Random Forest/best_random_forest_model.pkl'

with open(filename, 'rb') as file:
    loaded_model = pickle.load(file)

print(f"Model berhasil dimuat dari '{filename}'")

predictions_encoded = loaded_model.predict(df_new_encoded.to_numpy())

predictions_labels = []

label_mapping = {
    0: 'Berat (rawat inap keparahan 3)',
    1: 'Rawat Jalan',
    2: 'Ringan (rawat inap keparahan 1)',
    3: 'Sedang (rawat inap keparahan 2)'
}

for value in predictions_encoded:
  # Gunakan mapping untuk mendapatkan label string
  predictions_labels.append(label_mapping.get(value, 'Unknown')) # Tambahkan 'Unknown' untuk nilai yang tidak terduga


# Tambahkan hasil prediksi sebagai kolom baru di df_new_encoded
# Gunakan list predictions_labels yang berisi string
df_new['Predicted_FKL23'] = predictions_labels

# Tampilkan DataFrame dengan hasil prediksi
print("\nHasil prediksi ditambahkan ke DataFrame:")
display(df_new)

Model berhasil dimuat dari '/content/drive/MyDrive/Machine Learning Models/Supervised Learning/Classification/Random Forest/best_random_forest_model.pkl'

Hasil prediksi ditambahkan ke DataFrame:


,FKL10,FKL19,FKL19A,FKL30,Predicted_FKL23
0,RJTL,S-4-16-III,DIAGNOSIS SISTEM MUSKULOSKLETAL & JARINGAN PE...,1621 - Ophthalmoscopy;8911 - Tonometry;9511 - ...,Rawat Jalan
1,RJTL,W-1-30-III,GANGGUAN JARINGAN KONEKTIF BERAT,9904 - Transfusion of packed cells;9923 - Inje...,Rawat Jalan
2,RJTL,M-1-02-I,KECEDERAAN PEMBULUH DARAH OTAK NON SPESIFIK &...,9327 - Stretching of muscle or tendon;9339 - O...,Rawat Jalan
3,RJTL,A-4-14-I,PROSEDUR KECIL PADA MATA,8950 - Ambulatory cardiac monitoring;9059 - Ot...,Rawat Jalan
4,RITL,J-4-21-III,PROSEDUR KAKI SEDANG,9916 - Injection of antidote;9929 - Injection ...,Ringan (rawat inap keparahan 1)
...,...,...,...,...,...
95,RITL,G-4-21-II,PERSALINAN YANG SUKAR (FALSE LABOR),0391 - Injection of anesthetic into spinal can...,Berat (rawat inap keparahan 3)
96,RITL,K-3-14-0,PROSEDUR ANAL BERAT,"8907 - Consultation, described as comprehensiv...",Ringan (rawat inap keparahan 1)
97,RJTL,F-4-13-II,BATU URIN BERAT,5012 - Open biopsy of liver;9904 - Transfusion...,Rawat Jalan
98,RITL,O-6-13-III,TUMOR SISTEM HEPATOBILLIARI DAN PANCREAS RINGAN,8872 - Diagnostic ultrasound of heart;897 - Ge...,Berat (rawat inap keparahan 3)


In [ ]:
# Nama file CSV untuk menyimpan DataFrame dengan prediksi
output_csv_filename = '/content/drive/MyDrive/Machine Learning Datasets/UTS/synthetic_data_with_predictions.csv'

# Simpan DataFrame df_new_encoded ke file CSV
# index=False agar kolom index tidak ikut tersimpan di CSV
df_new.to_csv(output_csv_filename, index=False)

print(f"DataFrame dengan data sintetik dan prediksi berhasil disimpan ke '{output_csv_filename}'")

DataFrame dengan data sintetik dan prediksi berhasil disimpan ke '/content/drive/MyDrive/Machine Learning Datasets/UTS/synthetic_data_with_predictions.csv'
